In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [2]:
spaceX_founding = datetime(2002, 3, 14)
spaceX_first_launch = datetime(2008, 9, 28)
spaceX_first_test_landing = datetime(2013, 9, 29)
spaceX_first_soft_touch_down = datetime(2014, 4, 8)
spaceX_first_landing = datetime(2015, 12, 21)
space_race_start = datetime(1957, 8, 21)
first_moon_landing = datetime(1969, 7, 20)

In [3]:
launches = pd.read_json('../data/launch-library/launches.json', encoding='utf=8')

### Calculate new and total launches and their statuses

In [4]:
launches_per_month = pd.DataFrame(launches[['id', 'net', 'status']])
launches_per_month['year'] = pd.DatetimeIndex(launches['net']).year
launches_per_month['month'] = pd.DatetimeIndex(launches['net']).month
launches_per_month['day'] = 1
launches_per_month['year_month'] = pd.to_datetime(launches_per_month[['year', 'month', 'day']])
launches_per_month.drop(['year', 'month', 'day'], axis=1, inplace=True)

launches_per_month['new'] = 1
launches_per_month['new_success'] = launches_per_month['status'].apply(lambda s: int(s['abbrev']=='Success'))
launches_per_month['new_failure'] = launches_per_month['status'].apply(lambda s: int(s['abbrev']=='Failure'))
launches_per_month['new_partial'] = launches_per_month['status'].apply(lambda s: int(s['abbrev']=='Partial Failure'))
launches_per_month['new_tbd'] = launches_per_month['status'].apply(lambda s: s['abbrev']=='TBD')

launches_per_month['total'] = 0
launches_per_month['total_success'] = 0
launches_per_month['total_failure'] = 0
launches_per_month['total_partial'] = 0
launches_per_month['total_tbd'] = 0

launches_per_month = launches_per_month.groupby(['year_month']).sum(numeric_only=True).reset_index()

for (i, row) in launches_per_month.iterrows():
  if i == 0:
    launches_per_month.loc[i, 'total'] = launches_per_month.loc[i, 'new']
    launches_per_month.loc[i, 'total_success'] = launches_per_month.loc[i, 'new_success']
    launches_per_month.loc[i, 'total_failure'] = launches_per_month.loc[i, 'new_failure']
    launches_per_month.loc[i, 'total_partial'] = launches_per_month.loc[i, 'new_partial']
    launches_per_month.loc[i, 'total_tbd'] = launches_per_month.loc[i, 'new_tbd']
  else:
    launches_per_month.loc[i, 'total'] = launches_per_month.loc[i-1, 'total'].sum() + launches_per_month.loc[i, 'new']
    launches_per_month.loc[i, 'total_success'] = launches_per_month.loc[i-1, 'total_success'].sum() + launches_per_month.loc[i, 'new_success']
    launches_per_month.loc[i, 'total_failure'] = launches_per_month.loc[i-1, 'total_failure'].sum() + launches_per_month.loc[i, 'new_failure']
    launches_per_month.loc[i, 'total_partial'] = launches_per_month.loc[i-1, 'total_partial'].sum() + launches_per_month.loc[i, 'new_partial']
    launches_per_month.loc[i, 'total_tbd'] = launches_per_month.loc[i-1, 'total_tbd'].sum() + launches_per_month.loc[i, 'new_tbd']

launches_per_month

,year_month,new,new_success,new_failure,new_partial,new_tbd,total,total_success,total_failure,total_partial,total_tbd
0,1957-10-01,1,1,0,0,0,1,1,0,0,0
1,1957-11-01,1,1,0,0,0,2,2,0,0,0
2,1957-12-01,1,0,1,0,0,3,2,1,0,0
3,1958-02-01,2,1,1,0,0,5,3,2,0,0
4,1958-03-01,3,2,1,0,0,8,5,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...
823,2032-12-01,2,0,0,0,2,6763,6010,486,29,228
824,2034-12-01,1,0,0,0,1,6764,6010,486,29,229
825,2038-12-01,1,0,0,0,1,6765,6010,486,29,230
826,2039-12-01,1,0,0,0,1,6766,6010,486,29,231


# Rolling average of monthly rocket launches

In [5]:
passive_color = '#bed9ec'
success_color = '#9ce68a'
annotation_bgcolor = '#222'
annotation_style = {'bgcolor': annotation_bgcolor, 'bordercolor': annotation_bgcolor, 'borderwidth': 5}

x_value = launches_per_month['year_month']

first_moon_landing_epoch = (datetime.fromtimestamp(0) - first_moon_landing).total_seconds() * -1000

fig = px.scatter(
	launches_per_month,
	x='year_month',
	y='new',
  trendline='rolling',
	trendline_color_override='#f00',
	trendline_options={
    'window': 25,
  },
)

fig.data[0].opacity = 0.5

fig.update_layout({
  'template': 'plotly_dark',
  'title': 'Global Rocket Launches over time',
  'height': 600,
  'margin_b': 150,
  'margin_r': 150,
  'xaxis': {
    'range': ['1955','2023'],
    'title': 'Time'
  },
  'yaxis': {
    'side': 'right',
    'title': 'Launches per month',
  },
  'legend_x': 0,
})

fig.add_vline(
  x=spaceX_founding.timestamp() * 1000,
  annotation={
    'text': 'SpaceX founded',
    'font_color': passive_color,
    'y': 0.7,
    **annotation_style
  },
  annotation_position='top',
  line={'dash': 'longdash', 'color': passive_color}
)
fig.add_vline(
  x=spaceX_first_launch.timestamp() * 1000,
  annotation={
    'text': 'SpaceX first launch',
    'font_color': passive_color,
    'y': 0.75,
    **annotation_style
  },
  annotation_position='top',
  line={'dash': 'longdash', 'color': passive_color}
)
fig.add_vline(
  x=spaceX_first_landing.timestamp() * 1000,
  annotation={
    'text': 'SpaceX first landing',
    'font_color': success_color,
    'y': 0.8,
    **annotation_style
  },
  annotation_position='top',
  line={'dash': 'longdash', 'color': success_color}
)
fig.add_vline(
  x=first_moon_landing_epoch,
  annotation={
    'text': 'First moon landing',
    'font_color': passive_color,
    'y': 0.7,
    **annotation_style
  },
  annotation_position='top',
  line={'dash': 'longdash', 'color': passive_color}
)
fig.add_vrect(
  x0=space_race_start,
  x1=first_moon_landing,
  line_width=0,
  fillcolor=passive_color,
  opacity=0.1,
  annotation={
    'text': f'Space Race<br>{space_race_start.year} - {first_moon_landing.year}',
    'font_color': passive_color,
    'y': 0.6,
    **annotation_style
  },
  annotation_position='left'
)

fig.add_annotation(go.layout.Annotation(
  text=f'During the <b style="color: {passive_color}">Space Race</b> we saw an incredible increase in<br>rocket launches mainly by the <b>Soviet Space Program</b> and <b>NASA</b>.',
  align='left',
  showarrow=False,
  x=first_moon_landing_epoch,
  yref='paper',
  y=-0.15,
  yanchor='top',
  bgcolor=annotation_bgcolor,
  bordercolor=annotation_bgcolor,
  borderwidth=10
))
fig.add_annotation(go.layout.Annotation(
  text='Currently we are wittnessing an even steeper<br>increase in rockets launched than during the Space Race.',
  align='left',
  showarrow=False,
  x=spaceX_first_launch,
  yref='paper',
  y=-0.15,
  yanchor='top',
  bgcolor=annotation_bgcolor,
  bordercolor=annotation_bgcolor,
  borderwidth=10
))

fig.show()